In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BartForConditionalGeneration, AutoTokenizer, BartTokenizer
import torch.optim
import accelerate
import os
import json
from tqdm import tqdm
from time import time
from random import sample

In [2]:
data_root_path = os.path.join(os.getcwd(), 'IndicHeadlineGeneration/data/hi_IndicHeadlineGeneration_v1.0')
model_path = os.path.join(os.getcwd(), 'Models')

In [3]:
train_path = os.path.join(data_root_path,'hi_train.jsonl')
dev_path = os.path.join(data_root_path,'hi_dev.jsonl')
test_path = os.path.join(data_root_path,'hi_test.jsonl')

In [4]:
class CustomDataset(Dataset):
    def __init__(self, data_path, tokenizer, sample_size, max_length=512):
        self.data = self.load_data(data_path, sample_size)
        self.tokenizer = tokenizer
        self.max_length = max_length
        #self.sample_size = sample_size
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        item = self.data[idx]
        input_text = item['Document']
        target_text = item['Title']
        
        inputs = self.tokenizer.encode_plus(input_text, max_length=self.max_length,
                                            padding='max_length', truncation=True, return_tensors='pt'
                                           )
        
        targets = self.tokenizer.encode(target_text, max_length=self.max_length,
                                        padding='max_length', truncation=True, return_tensors='pt'
                                        )
        
        return {'input_ids': inputs['input_ids'].squeeze(),
                'attention_mask': inputs['attention_mask'].squeeze(),
                'labels': targets.squeeze()
               }
    def load_data(self, data_path, sample_size):
        '''with open(data_path, 'r',encoding="utf8") as file:
            data = json.load(file)'''
        data = []
        with open(data_path, 'r', encoding="utf8") as file:
            for line in file:
                data.append(json.loads(line))
        return sample(data, sample_size)

In [5]:
model_name = "facebook/bart-base"
tokenizer = BartTokenizer.from_pretrained(model_name)

In [6]:
train_dataset = CustomDataset(train_path, tokenizer, 4000)
dev_dataset = CustomDataset(dev_path, tokenizer, 600)
test_dataset = CustomDataset(test_path, tokenizer, 1000)

In [7]:
len(train_dataset)

4000

In [8]:
train_loader = DataLoader(train_dataset, batch_size=25, shuffle=True)
val_loader = DataLoader(dev_dataset, batch_size=25, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=25, shuffle=False)

In [9]:
def Fine_Tune(train_loader, val_loader, num_epochs, num_layers_to_finetune):
    model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
    
    for param in model.parameters():
        param.requires_grad = False
    
    for i in range(1, num_layers_to_finetune + 1):
        for param in model.model.encoder.layers[-i].parameters():
            param.requires_grad = True
        for param in model.model.decoder.layers[-i].parameters():
            param.requires_grad = True
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(device)
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=7e-5, weight_decay=3e-6)
    for epoch in tqdm(range(num_epochs)):
        model.train()
        print("Training phase")
        #for idx, batch in enumerate(train_loader):
        for batch in tqdm(train_loader):
            #time0 = time()
            #print(f"batch: {idx+1} starts")
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            #time1 = time()
            #print(f"batch: {idx+1} ends\ntime taken: {time1-time0} seconds")
            #print(f"time taken: {time1-time0} seconds")
        
        model.eval()
        print("Validation phase")
        with torch.no_grad():
            total_val_loss = 0.0
            for val_batch in tqdm(val_loader):
                input_ids = val_batch['input_ids'].to(device)
                attention_mask = val_batch['attention_mask'].to(device)
                labels = val_batch['labels'].to(device)
                val_output = model(input_ids, attention_mask=attention_mask, labels=labels)
                total_val_loss += val_output.loss.item()
            average_val_loss = total_val_loss / len(val_loader)
            
        print(f'Epoch: {epoch+1}/{num_epochs}, Validation Loss: {average_val_loss}')
    os.makedirs(model_path, exist_ok=True)
    model.save_pretrained(model_path, from_pt=True)
    torch.save(model.state_dict(), os.path.join(model_path, 'pytorch_model.bin'))

In [10]:
def Generate_title(val_loader):
    model = BartForConditionalGeneration.from_pretrained(model_path)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    with torch.no_grad():
        for val_batch in val_loader:
            input_ids = val_batch['input_ids'].to(device)
            attention_mask = val_batch['attention_mask'].to(device)
            labels = val_batch['labels'].to(device)
            generated_ids = model.generate(input_ids, attention_mask=attention_mask,
                                           max_length=60, num_beams=2, repetition_penalty=2.0,
                                           length_penalty=2.0, early_stopping=True
                                          )
            generated_title = tokenizer.decode(generated_ids[0])
            input_text = tokenizer.decode(input_ids[0])
            actual_title = tokenizer.decode(labels[0])
            
            print(f'Input Text: {input_text}\nGenerated Title: {generated_title}\nActual Title: {actual_title}')
            print('\n'+'='*50+'\n')

In [11]:
Fine_Tune(train_loader, val_loader, 5, 2)

cuda


  0%|                                                                                                                                                                                                                                                   | 0/5 [00:00<?, ?it/s]

Training phase



  0%|                                                                                                                                                                                                                                                 | 0/160 [00:00<?, ?it/s]C:\Users\udaya\miniconda3\envs\torch\Lib\site-packages\transformers\models\bart\modeling_bart.py:590: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(

 17%|███████████████████████████████████████▏                                                                                                                                                                                                | 27/160 [03:18<16:52,  7.62s/it]


 35%|█████████████████████████████████████████████████████████████████████████████████▏                                                                                                                                                      | 56/160 [07:31<16:56,  9.78s/it]


...

 90%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                       | 144/160 [21:00<02:22,  8.92s/it]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 160/160 [23:24<00:00,  8.78s/it]


Validation phase



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [2:04:02<00:00, 1488.43s/it]
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


Epoch: 5/5, Validation Loss: 3.6714065174261727


In [12]:
Generate_title(test_loader)

Input Text: <s>भारत में 75 सबसे अधिक मूल्यवान ब्रांडों की भीड़ में सबको पीछे छोड़ते हुए एचडीएफसी बैंक पिछले साल की तरह पहले स्थान पर कायम है। दूसरे और तीसरे स्थान पर भी पुराने ब्रांडों ने ही अपना कब्जा बनाए रखा है। सार्वजनिक क्षेत्र की बीमा कंपनी भारतीय जीवन बीमा निगम (एलआईसी) और आईटी क्षेत्र की दिग्गज कंपनी टाटा कंसल्टेंसी सर्विसेज (टीसीए�</s>
Generated Title: </s><s>भारत में 75 सबसे अधिक पहुंची एचडीएफस,</s>
Actual Title: <s>एचडीएफसी बैंक पहला और एलआईसी दूसरा सबसे मूल्यवान ब्रांड</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

Input Text: <s>एलऐंड़टी टेक्नोलॉजी सर्विसेज के ऑफर फोर सेल (ओएफएस) को गुरुवार को निवेशकों की बहुत अच्छी प्रतिक्रिया नहीं मिली और खुदरा श्रेणी में महज 2.33 फीसदी आवेदन मिले। एक दिन पहले संस्थागत निवेशकों की श्रेणी में भी महज 57 फीसदी आवेदन मिले थे। सूत्रों ने कहा कि निवेशकों की मांग कम कीमत पर थी, लेकिन कंपनी ने शेयर बिक्री की कीमत आक्राम</s>
Generated Title: </s><s>एलऐंड़टी टेक्नोलॉजी, सरुविसे-� के</s>
Actual Title: <s>एलऐंडटी टेक का ओएफएस, खुदरा में कम आवेदन</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

Input Text: <s>'तू आशिकी' सीरियल यूथ के बीच काफी पॉपुलर हो गया है।शो के लीड पेयर पंक्ति और अहान की जोड़ी दर्शक खूब पसंद कर रहे हैं।अहान-पंक्ति नई दिल्लीः 16 साल की टीवी एक्ट्रेस जन्नत जुबैर रहमानी को उनके सीरियल 'तू आशिकी' के को-स्टार ऋत्विक अरोड़ा को किस करने को कहा गया।लेकिन जन्नत की मां इस बात के लिए राजी नहीं हुई है।एक एंटरटेनमेंट वेबसाइ�</s>
Generated Title: </s><s>'तू आशिकी' पेयर पंका�ति और,�हान को जोड</s>
Actual Title: <s>पंक्ति और अहान के बीच होना था किसिंग सीन, लेकिन इस वजह से मां ने लगा दी रोक</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

Input Text: <s>प्रधानमंत्री नरेंद्र मोदी के कामकाज की शैली में पिछले कुछ सप्ताह में बड़ा बदलाव देखने को मिला है। यह बदलाव मीडिया के साथ मेलजोल के रूप में नजर आया है। आलोचक मोदी के बदले रुख के लिए जिम्मेदार वजहें तलाश कर ही लेंगे लेकिन उससे यह तथ्य नहीं छिपेगा कि बदलाव जरूर आया है और यह बदलाव बतौर प्रधानमंत्री उनके तथा उनकी सरकार के लिए कई अहम नि</s>
Generated Title: </s><s>प्रधानमंत् की शैली,�ें बदलाव मोदी</s><pad><pad><pad>
Actual Title: <s>मोदी सरकार के मीडिया से लगाव को न समझें दिल में आया बदलाव</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

Input Text: <s>जीका वायरस के फिजी में इस साल जनवरी से तीन अगस्त के बीच 34 मामले सामने आए हैं।सुवाः जीका वायरस के फिजी में इस साल जनवरी से तीन अगस्त के बीच 34 मामले सामने आए हैं।स्वास्थ्य मंत्री जोन उसामेट ने गुरुवार को बताया कि जीका के अलावा, देश में इसी अवधि के दौरान डेंगू के 623, चिकनगुनिया के 41, टाइफाइड के 225 और लेप्टोस्पायरोसिस के 430 मामले सामने आए हैं।समा�</s>
Generated Title: </s><s>जीका वायरस के अगस्त में बुच 34 मामले, सा�</s>
Actual Title: <s>जीका वायरस के फिजी में 34 मामले</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

Input Text: <s>प्रधानमंत्री मनमोहन सिंह ब्रिक शिखर सम्मेलन में भाग लेने के लिए मंगलवार को चीन पहुंच गए हैं। इस सम्मेलन में दक्षिण अफ्रीका को ब्रिक्स के पांचवें सदस्य के रूप में औपचारिक तौर पर शामिल किया जाएगा। फिलहाल ब्राजील, रूस, भारत और चीन ब्रिक्स के सदस्य देश हैं। सान्या हवाई अड्डïे पर चीन के सहायक विदेशी मंत्री वु हेलांग ने प्रधानमंत</s>
Generated Title: </s><s>मंगलवार को ब्रिक शि-�र, पहुच गए हैं। �</s>
Actual Title: <s>ब्रिक्स में रहेंगे विकास और विवाद के मुद्दे</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

Input Text: <s>ऑस्ट्रेलिया के खिलाफ पहला वनडे मैच पूरी एकदिवसीय सीरीज़ के भाग्य को निर्धारित करने की दिशा में बहुत महत्वपूर्ण रोल प्ले करेगा।अकसर यह सीरीज़ का पहला मैच ही होता ऑस्ट्रेलिया के खिलाफ पहला वनडे मैच पूरी एकदिवसीय सीरीज़ के भाग्य को निर्धारित करने की दिशा में बहुत महत्वपूर्ण रोल प्ले करेगा।अकसर यह सीरीज़ का पहला मैच ही होता ह</s>
Generated Title: </s><s>एकदिवसीय से भाग्य, को नि'�् '�ारि', पहला</s>
Actual Title: <s>तेज़ गेंदबाज़ों के प्रदर्शन से निश्चित होगा आज के मैच का परिणाम</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

Input Text: <s>केंद्र सरकार ने महाराष्ट्र, मध्य प्रदेश, कर्नाटक और राजस्थान में ओलावृष्टि और बेमौसम बारिश से प्रभावित वानिकी फसलों को फिर से लगाने के लिए 352 करोड़ रुपये की सहायता मंजूर की है। यह जानकारी आज एक सरकारी सूत्र ने कही। सूत्र ने बताया कि इसके अलावा कर्नाटक के उन किसानों के लिए 92 करोड़ रुपये सहायता मंजूर की गई है जिनके मकान और रबी की फसलों �</s>
Generated Title: </s><s>केंद्र सरकार, ने मिलों की बताया</s><pad><pad><pad><pad><pad><pad><pad><pad><pad>
Actual Title: <s>चार राज्यों को 352 करोड़ रुपये की सहायता</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

Input Text: <s> आयुष्मान और भूमि ने एक वीडियो भी जारी किया है जिसमें ये दिखाया गया है कि भारत में लोग सेक्स पर किस तरह बात करते हैं।नई दिल्लीः'बरेली की बर्फी'के बाद अभिनेता आयुष्मान खुराना की अगली फिल्म'शुभ मंगल सावधान'कल यानी 1 सितंबर को रिलीज होने जा रही है।फिल्म में आयुष्मान के साथ 'दम लगा के हईशा' में उनके साथ दिखी एक्ट्रेस भूमि पेडनेक र लीड र�</s>
Generated Title: </s><s>आयुष्मान और भूमि ने किस तरह बात, जारी �</s>
Actual Title: <s>शादी से पहले सेक्स पर बोले आयुष्मान खुराना</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

Input Text: <s>आज अमेरिका में राष्ट्रपति चुनाव के लिए वोटिंग जारी है।जनवरी महीने से ही चुनावी सरगर्मियां तेज हो गई थी।दुनिया के सबसे ताकतवर मुल्क अमेरिका में राष्ट्रपति चयन की प्रक्रिया भारत के मुकाबले थोड़ा अलग और खास है।अमेरिका का राष्ट्रपति नई दिल्लीः आज अमेरिका में राष्ट्रपति चुनाव के लिए वोटिंग जारी है।राष्ट्रपति पद के लिए डेमोक�</s>
Generated Title: </s><s>आज अमेरिका से ही चुनावी, प्य�कोट्'�प�</s>
Actual Title: <s>अमेरिका चुन रहा है अगली सरकार, जानिए यूएस में कैसे बनता है कोई राष्ट्रपति?</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa